# 0- Imports

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

try:
    from amplpy import AMPL, tools
except ImportError:
    # Caso ocorra um erro ao importar, instala amplpy e tenta importar novamente
    %pip install amplpy
    from amplpy import AMPL, tools

In [2]:
licenca ="default"


ampl = tools.ampl_notebook(
    modules=["highs", "gcg", "gurobi", "cplex"],  # Módulos a serem instalados
    license_uuid=licenca,  # Licença a ser utilizada
    g=globals()  # Instancia o objeto AMPL e registra os comandos mágicos
)

# 1- Funções

In [3]:
def transforma_obj_to_str(df):
    var_obj = list(df.select_dtypes('object').columns)

    df[var_obj] = df[var_obj].astype('string')

    return df

# 2- Base

## 2.1 - Produtos

In [4]:
df_produtos = pd.read_excel('../../01-Data/extra/dados.xlsx', sheet_name='produtos')
df_produtos

,instancia,classe_onda,tipo_buffer,sku,tipo_caixa_padrao,item_pai,codigo_cor,sku_unit_kg,sku_unit_m3,inner
0,instancia-1,classe_onda-1,tipo_buffer-1,sku-12155,CARTON_M,itempai-3488,cor-36,0.100,0.001289,1
1,instancia-1,classe_onda-1,tipo_buffer-1,sku-12156,CARTON_M,itempai-3488,cor-36,0.100,0.002537,1
2,instancia-1,classe_onda-1,tipo_buffer-1,sku-12157,CARTON_M,itempai-3488,cor-36,0.140,0.001098,1
3,instancia-1,classe_onda-1,tipo_buffer-1,sku-12158,CARTON_M,itempai-3488,cor-36,0.127,0.002070,1
4,instancia-1,classe_onda-1,tipo_buffer-1,sku-12159,CARTON_M,itempai-3488,cor-36,0.120,0.001840,1
...,...,...,...,...,...,...,...,...,...,...
43866,instancia-5,classe_onda-9,tipo_buffer-1,sku-9749,CARTON_S,itempai-2852,cor-989,0.260,0.000584,1
43867,instancia-5,classe_onda-9,tipo_buffer-1,sku-977,CARTON_S,itempai-248,cor-989,0.240,0.000374,1
43868,instancia-5,classe_onda-9,tipo_buffer-1,sku-980,CARTON_S,itempai-250,cor-23,0.260,0.001000,1
43869,instancia-5,classe_onda-9,tipo_buffer-1,sku-981,CARTON_S,itempai-251,cor-989,0.285,0.000812,1


In [5]:
df_produtos = transforma_obj_to_str(df_produtos)

In [6]:
df_produtos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43871 entries, 0 to 43870
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   instancia          43871 non-null  string 
 1   classe_onda        43871 non-null  string 
 2   tipo_buffer        43871 non-null  string 
 3   sku                43871 non-null  string 
 4   tipo_caixa_padrao  43871 non-null  string 
 5   item_pai           43871 non-null  string 
 6   codigo_cor         43871 non-null  string 
 7   sku_unit_kg        43871 non-null  float64
 8   sku_unit_m3        43871 non-null  float64
 9   inner              43871 non-null  int64  
dtypes: float64(2), int64(1), string(7)
memory usage: 3.3 MB


In [7]:
df_produtos['chave_produtos'] = df_produtos['instancia'] + '_' + df_produtos['classe_onda'] + '_' + df_produtos['tipo_buffer'] \
                            + df_produtos['sku']

In [8]:
df_produtos['chave_caixas'] = df_produtos['instancia'] + '_' + df_produtos['classe_onda'] + '_' + df_produtos['tipo_buffer'] \
                            + '_' + df_produtos['tipo_caixa_padrao']

In [42]:
df_produtos[['instancia', 'classe_onda', 'tipo_buffer', 'sku']].value_counts()

instancia    classe_onda    tipo_buffer    sku      
instancia-1  classe_onda-1  tipo_buffer-1  sku-12155    1
instancia-4  classe_onda-2  tipo_buffer-2  sku-4164     1
                                           sku-4150     1
                                           sku-4151     1
                                           sku-4152     1
                                                       ..
instancia-2  classe_onda-2  tipo_buffer-2  sku-7485     1
                                           sku-7488     1
                                           sku-7489     1
                                           sku-7490     1
instancia-5  classe_onda-9  tipo_buffer-1  sku-982      1
Name: count, Length: 43871, dtype: int64

## 2.2- Caixas

In [9]:
df_caixas = pd.read_excel('../../01-Data/extra/dados.xlsx', sheet_name='tipos caixas')
df_caixas

,instancia,tipo_buffer,classe_onda,tipo_caixa,caixa_cap_pecas,caixa_cap_kg,caixa_cap_m3,caixa_m3_externo
0,instancia-1,tipo_buffer-1,classe_onda-1,CARTON_M,170,23,0.125392,0.075995
1,instancia-1,tipo_buffer-1,classe_onda-10,CARTON_M,170,23,0.125392,0.075995
2,instancia-1,tipo_buffer-1,classe_onda-11,CARTON_M,170,23,0.125392,0.075995
3,instancia-1,tipo_buffer-1,classe_onda-2,CARTON_M,170,23,0.125392,0.075995
4,instancia-1,tipo_buffer-1,classe_onda-3,CARTON_L,170,23,0.074563,0.093204
...,...,...,...,...,...,...,...,...
136,instancia-5,tipo_buffer-2,classe_onda-3,CARTON_M,512,23,0.125392,0.075995
137,instancia-5,tipo_buffer-2,classe_onda-4,CARTON_M,512,23,0.113993,0.075995
138,instancia-5,tipo_buffer-2,classe_onda-5,CARTON_L,512,23,0.074563,0.093204
139,instancia-5,tipo_buffer-2,classe_onda-5,CARTON_M,512,23,0.125392,0.075995


In [10]:
df_caixas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141 entries, 0 to 140
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   instancia         141 non-null    object 
 1   tipo_buffer       141 non-null    object 
 2   classe_onda       141 non-null    object 
 3   tipo_caixa        141 non-null    object 
 4   caixa_cap_pecas   141 non-null    int64  
 5   caixa_cap_kg      141 non-null    int64  
 6   caixa_cap_m3      141 non-null    float64
 7   caixa_m3_externo  141 non-null    float64
dtypes: float64(2), int64(2), object(4)
memory usage: 8.9+ KB


In [11]:
df_caixas = transforma_obj_to_str(df_caixas)

In [12]:
df_caixas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141 entries, 0 to 140
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   instancia         141 non-null    string 
 1   tipo_buffer       141 non-null    string 
 2   classe_onda       141 non-null    string 
 3   tipo_caixa        141 non-null    string 
 4   caixa_cap_pecas   141 non-null    int64  
 5   caixa_cap_kg      141 non-null    int64  
 6   caixa_cap_m3      141 non-null    float64
 7   caixa_m3_externo  141 non-null    float64
dtypes: float64(2), int64(2), string(4)
memory usage: 8.9 KB


In [13]:
df_caixas.columns

Index(['instancia', 'tipo_buffer', 'classe_onda', 'tipo_caixa',
       'caixa_cap_pecas', 'caixa_cap_kg', 'caixa_cap_m3', 'caixa_m3_externo'],
      dtype='object')

In [14]:
df_caixas['chave_caixas'] = df_caixas['instancia'] + '_' + df_caixas['classe_onda'] + '_' + df_caixas['tipo_buffer'] \
                            + '_' + df_caixas['tipo_caixa']

In [15]:
df_caixas['chave_caixas'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 141 entries, 0 to 140
Series name: chave_caixas
Non-Null Count  Dtype 
--------------  ----- 
141 non-null    string
dtypes: string(1)
memory usage: 1.2 KB


In [16]:
df_produtos_cx = df_produtos.merge(df_caixas[['chave_caixas', 'caixa_cap_pecas', 'caixa_cap_kg', 
                                              'caixa_cap_m3', 'caixa_m3_externo']], on='chave_caixas', how='left')
df_produtos_cx

,instancia,classe_onda,tipo_buffer,sku,tipo_caixa_padrao,item_pai,codigo_cor,sku_unit_kg,sku_unit_m3,inner,chave_produtos,chave_caixas,caixa_cap_pecas,caixa_cap_kg,caixa_cap_m3,caixa_m3_externo
0,instancia-1,classe_onda-1,tipo_buffer-1,sku-12155,CARTON_M,itempai-3488,cor-36,0.100,0.001289,1,instancia-1_classe_onda-1_tipo_buffer-1sku-121...,instancia-1_classe_onda-1_tipo_buffer-1_CARTON_M,170,23,0.125392,0.075995
1,instancia-1,classe_onda-1,tipo_buffer-1,sku-12156,CARTON_M,itempai-3488,cor-36,0.100,0.002537,1,instancia-1_classe_onda-1_tipo_buffer-1sku-121...,instancia-1_classe_onda-1_tipo_buffer-1_CARTON_M,170,23,0.125392,0.075995
2,instancia-1,classe_onda-1,tipo_buffer-1,sku-12157,CARTON_M,itempai-3488,cor-36,0.140,0.001098,1,instancia-1_classe_onda-1_tipo_buffer-1sku-121...,instancia-1_classe_onda-1_tipo_buffer-1_CARTON_M,170,23,0.125392,0.075995
3,instancia-1,classe_onda-1,tipo_buffer-1,sku-12158,CARTON_M,itempai-3488,cor-36,0.127,0.002070,1,instancia-1_classe_onda-1_tipo_buffer-1sku-121...,instancia-1_classe_onda-1_tipo_buffer-1_CARTON_M,170,23,0.125392,0.075995
4,instancia-1,classe_onda-1,tipo_buffer-1,sku-12159,CARTON_M,itempai-3488,cor-36,0.120,0.001840,1,instancia-1_classe_onda-1_tipo_buffer-1sku-121...,instancia-1_classe_onda-1_tipo_buffer-1_CARTON_M,170,23,0.125392,0.075995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44111,instancia-5,classe_onda-9,tipo_buffer-1,sku-9749,CARTON_S,itempai-2852,cor-989,0.260,0.000584,1,instancia-5_classe_onda-9_tipo_buffer-1sku-974...,instancia-5_classe_onda-9_tipo_buffer-1_CARTON_S,170,23,0.029816,0.035078
44112,instancia-5,classe_onda-9,tipo_buffer-1,sku-977,CARTON_S,itempai-248,cor-989,0.240,0.000374,1,instancia-5_classe_onda-9_tipo_buffer-1sku-977...,instancia-5_classe_onda-9_tipo_buffer-1_CARTON_S,170,23,0.029816,0.035078
44113,instancia-5,classe_onda-9,tipo_buffer-1,sku-980,CARTON_S,itempai-250,cor-23,0.260,0.001000,1,instancia-5_classe_onda-9_tipo_buffer-1sku-980...,instancia-5_classe_onda-9_tipo_buffer-1_CARTON_S,170,23,0.029816,0.035078
44114,instancia-5,classe_onda-9,tipo_buffer-1,sku-981,CARTON_S,itempai-251,cor-989,0.285,0.000812,1,instancia-5_classe_onda-9_tipo_buffer-1sku-981...,instancia-5_classe_onda-9_tipo_buffer-1_CARTON_S,170,23,0.029816,0.035078


## 2.3- Lojas

In [17]:
df_loja = pd.read_excel('../../01-Data/extra/dados.xlsx', sheet_name='lojas')
df_loja

,instancia,rota,loja,cap_loja_m3
0,instancia-1,rota-1,loja-209,NaN
1,instancia-1,rota-17,loja-259,NaN
2,instancia-1,rota-17,loja-83,15.0
3,instancia-1,rota-24,loja-112,10.0
4,instancia-1,rota-25,loja-35,22.0
...,...,...,...,...
355,instancia-5,rota-69,loja-89,NaN
356,instancia-5,rota-7,loja-118,16.5
357,instancia-5,rota-7,loja-226,13.8
358,instancia-5,rota-7,loja-273,NaN


In [18]:
df_loja.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 360 entries, 0 to 359
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   instancia    360 non-null    object 
 1   rota         360 non-null    object 
 2   loja         360 non-null    object 
 3   cap_loja_m3  148 non-null    float64
dtypes: float64(1), object(3)
memory usage: 11.4+ KB


In [19]:
df_loja = transforma_obj_to_str(df_loja)

In [20]:
df_loja.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 360 entries, 0 to 359
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   instancia    360 non-null    string 
 1   rota         360 non-null    string 
 2   loja         360 non-null    string 
 3   cap_loja_m3  148 non-null    float64
dtypes: float64(1), string(3)
memory usage: 11.4 KB


## 2.4- Rotas

In [21]:
df_rotas = pd.read_excel('../../01-Data/extra/dados.xlsx', sheet_name='rotas')
df_rotas

,instancia,rota,cap_rota_m3
0,instancia-1,rota-1,46
1,instancia-1,rota-17,15
2,instancia-1,rota-24,31
3,instancia-1,rota-25,46
4,instancia-1,rota-30,46
...,...,...,...
80,instancia-5,rota-5,15
81,instancia-5,rota-6,31
82,instancia-5,rota-68,31
83,instancia-5,rota-69,92


In [22]:
df_rotas = transforma_obj_to_str(df_rotas)

In [23]:
df_rotas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85 entries, 0 to 84
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   instancia    85 non-null     string
 1   rota         85 non-null     string
 2   cap_rota_m3  85 non-null     int64 
dtypes: int64(1), string(2)
memory usage: 2.1 KB


## 2.5- Ordens de produção

In [24]:
df_ord_producao = pd.read_csv('../../01-Data/extra/ordens producao.csv')
df_ord_producao

,instancia,alocacao_id,alocacao_datahora,tipo_buffer,loja,sku,pecas
0,instancia-1,alocacao-1,2024-05-29 19:07:06.923,tipo_buffer-1,loja-296,sku-77,7
1,instancia-1,alocacao-1000,2024-05-23 11:12:43.127,tipo_buffer-1,loja-58,sku-10540,14
2,instancia-1,alocacao-10000,2024-06-14 18:05:48.893,tipo_buffer-2,loja-209,sku-11595,1
3,instancia-1,alocacao-10001,2024-06-14 18:05:42.413,tipo_buffer-2,loja-209,sku-11040,1
4,instancia-1,alocacao-10002,2024-06-14 18:05:46.887,tipo_buffer-2,loja-209,sku-6714,1
...,...,...,...,...,...,...,...
228578,instancia-5,alocacao-9633,2024-06-14 17:01:17.647,tipo_buffer-1,loja-15,sku-3806,2
228579,instancia-5,alocacao-9633,2024-06-14 17:01:17.647,tipo_buffer-1,loja-188,sku-3806,4
228580,instancia-5,alocacao-9633,2024-06-14 17:01:17.647,tipo_buffer-1,loja-94,sku-3806,2
228581,instancia-5,alocacao-964,2024-05-22 13:25:20.723,tipo_buffer-2,loja-15,sku-2720,2


In [25]:
df_ord_producao = transforma_obj_to_str(df_ord_producao)

In [26]:
df_ord_producao['alocacao_datahora'] = pd.to_datetime(df_ord_producao['alocacao_datahora'])

In [27]:
df_ord_producao.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 228583 entries, 0 to 228582
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   instancia          228583 non-null  string        
 1   alocacao_id        228583 non-null  string        
 2   alocacao_datahora  228583 non-null  datetime64[ns]
 3   tipo_buffer        228583 non-null  string        
 4   loja               228583 non-null  string        
 5   sku                228583 non-null  string        
 6   pecas              228583 non-null  int64         
dtypes: datetime64[ns](1), int64(1), string(5)
memory usage: 12.2 MB


In [28]:
df_ord_producao['chave_produtos_aloc'] = df_ord_producao['instancia'] + '_' + df_ord_producao['alocacao_id'] + '_' + df_ord_producao['loja'] \
                            + df_ord_producao['sku']

In [52]:
df_ord_producao['chave_produtos_aloc'].value_counts()

chave_produtos_aloc
instancia-1_alocacao-1_loja-296sku-77           1
instancia-1_alocacao-10009_loja-209sku-12199    1
instancia-1_alocacao-10098_loja-209sku-5056     1
instancia-1_alocacao-10010_loja-209sku-3501     1
instancia-1_alocacao-10000_loja-209sku-11595    1
                                               ..
instancia-5_alocacao-9633_loja-138sku-3806      1
instancia-5_alocacao-9633_loja-146sku-3806      1
instancia-5_alocacao-9633_loja-15sku-3806       1
instancia-5_alocacao-9633_loja-188sku-3806      1
instancia-5_alocacao-964_loja-37sku-2720        1
Name: count, Length: 228583, dtype: Int64

In [56]:
pd.DataFrame(df_ord_producao[['instancia', 'alocacao_id', 'loja','sku']].value_counts()).head(20)

count
instancia   alocacao_id    loja     sku            
instancia-1 alocacao-1     loja-296 sku-77        1
instancia-4 alocacao-18285 loja-87  sku-1084      1
            alocacao-18283 loja-87  sku-919       1
                                    sku-920       1
            alocacao-18284 loja-87  sku-1903      1
                                    sku-1936      1
                                    sku-1968      1
                                    sku-2483      1
                                    sku-3568      1
                                    sku-3569      1
                                    sku-3570      1
                                    sku-3571      1
            alocacao-18285 loja-87  sku-1083      1
                                    sku-1085      1
            alocacao-18282 loja-87  sku-2116      1
            alocacao-18285 loja-87  sku-1086      1
                                    sku-1087      1
                                    sku-1088      1
                                    sku-1089      1
                                    sku-1220      1

## 2.6- Saída algoritmo Renner

In [29]:
df_output = pd.read_csv('../../01-Data/extra/dados_output.csv')
df_output

,cap_loja_m3,cap_rota_m3,caixa_cap_pecas,caixa_cap_kg,caixa_cap_m3,caixa_m3_externo,alocacao_datahora,sku_unit_kg,sku_unit_m3,inner,...,alocacao_id,loja,rota,caixa_id,sku,item_pai,codigo_cor,tipo_buffer,classe_onda,tipo_caixa
0,NaN,31.0,170.0,23.0,0.125392,0.075995,2024-06-12 11:10:28.833,0.550,0.017008,1,...,alocacao-7300,loja-242,rota-54,caixa-4472,sku-6694,itempai-1998,cor-783,tipo_buffer-1,classe_onda-1,CARTON_M
1,NaN,31.0,170.0,23.0,0.125392,0.075995,2024-06-12 11:10:28.833,0.550,0.017008,1,...,alocacao-7300,loja-242,rota-54,caixa-4473,sku-6694,itempai-1998,cor-783,tipo_buffer-1,classe_onda-1,CARTON_M
2,NaN,31.0,170.0,23.0,0.125392,0.075995,2024-06-12 11:10:28.833,0.550,0.017008,1,...,alocacao-7300,loja-242,rota-54,caixa-4474,sku-6694,itempai-1998,cor-783,tipo_buffer-1,classe_onda-1,CARTON_M
3,NaN,31.0,170.0,23.0,0.125392,0.075995,2024-06-12 11:10:28.840,0.530,0.005107,1,...,alocacao-7295,loja-242,rota-54,caixa-4472,sku-6695,itempai-1998,cor-783,tipo_buffer-1,classe_onda-1,CARTON_M
4,NaN,31.0,170.0,23.0,0.125392,0.075995,2024-06-12 11:10:28.840,0.530,0.005107,1,...,alocacao-7295,loja-242,rota-54,caixa-4473,sku-6695,itempai-1998,cor-783,tipo_buffer-1,classe_onda-1,CARTON_M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265133,11.0,15.0,512.0,23.0,0.091194,0.075995,2024-06-10 09:16:30.673,0.345,0.005022,1,...,alocacao-5755,loja-96,rota-10,caixa-19140,sku-2819,itempai-847,cor-741,tipo_buffer-2,classe_onda-6,CARTON_M
265134,11.0,15.0,512.0,23.0,0.091194,0.075995,2024-06-10 09:16:30.743,0.300,0.004437,1,...,alocacao-5754,loja-96,rota-10,caixa-19140,sku-2820,itempai-847,cor-741,tipo_buffer-2,classe_onda-6,CARTON_M
265135,11.0,15.0,512.0,23.0,0.091194,0.075995,2024-06-10 09:16:31.377,0.315,0.004167,1,...,alocacao-5757,loja-96,rota-10,caixa-19140,sku-2817,itempai-847,cor-741,tipo_buffer-2,classe_onda-6,CARTON_M
265136,11.0,15.0,512.0,23.0,0.091194,0.075995,2024-06-10 09:16:31.377,0.325,0.004948,1,...,alocacao-5756,loja-96,rota-10,caixa-19140,sku-2818,itempai-847,cor-741,tipo_buffer-2,classe_onda-6,CARTON_M


In [30]:
df_output.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265138 entries, 0 to 265137
Data columns (total 22 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   cap_loja_m3        179982 non-null  float64
 1   cap_rota_m3        265138 non-null  float64
 2   caixa_cap_pecas    265138 non-null  float64
 3   caixa_cap_kg       265138 non-null  float64
 4   caixa_cap_m3       265138 non-null  float64
 5   caixa_m3_externo   265138 non-null  float64
 6   alocacao_datahora  265138 non-null  object 
 7   sku_unit_kg        265138 non-null  float64
 8   sku_unit_m3        265138 non-null  float64
 9   inner              265138 non-null  int64  
 10  pecas              265138 non-null  int64  
 11  instancia          265138 non-null  object 
 12  alocacao_id        265138 non-null  object 
 13  loja               265138 non-null  object 
 14  rota               265138 non-null  object 
 15  caixa_id           265138 non-null  object 
 16  sk

In [31]:
df_output = transforma_obj_to_str(df_output)

In [32]:
df_output['alocacao_datahora'] = pd.to_datetime(df_output['alocacao_datahora'])

In [43]:
df_output['chave_produtos'] = df_output['instancia'] + '_' + df_output['classe_onda'] + '_' + df_output['tipo_buffer'] \
                            + '_' + df_output['sku'] + '_' + df_output['tipo_caixa']

In [44]:
df_output['chave_produtos'].value_counts()

chave_produtos
instancia-3_classe_onda-3_tipo_buffer-2_sku-11930_CARTON_M    98
instancia-3_classe_onda-3_tipo_buffer-2_sku-11929_CARTON_M    90
instancia-3_classe_onda-5_tipo_buffer-2_sku-10237_CARTON_M    88
instancia-3_classe_onda-3_tipo_buffer-2_sku-11928_CARTON_M    88
instancia-3_classe_onda-5_tipo_buffer-2_sku-10238_CARTON_M    84
                                                              ..
instancia-3_classe_onda-2_tipo_buffer-1_sku-2129_CARTON_M      1
instancia-3_classe_onda-2_tipo_buffer-1_sku-2128_CARTON_M      1
instancia-3_classe_onda-2_tipo_buffer-1_sku-6797_CARTON_M      1
instancia-3_classe_onda-2_tipo_buffer-1_sku-7129_CARTON_M      1
instancia-5_classe_onda-6_tipo_buffer-2_sku-4369_CARTON_M      1
Name: count, Length: 43871, dtype: Int64

In [45]:
df_output.query('chave_produtos == "instancia-3_classe_onda-3_tipo_buffer-2_sku-11930_CARTON_M"')

,cap_loja_m3,cap_rota_m3,caixa_cap_pecas,caixa_cap_kg,caixa_cap_m3,caixa_m3_externo,alocacao_datahora,sku_unit_kg,sku_unit_m3,inner,...,loja,rota,caixa_id,sku,item_pai,codigo_cor,tipo_buffer,classe_onda,tipo_caixa,chave_produtos
154629,NaN,300.0,512.0,23.0,0.125392,0.075995,2024-06-14 11:17:56.217,0.5,0.005706,1,...,loja-1,rota-62,caixa-10120,sku-11930,itempai-3234,cor-718,tipo_buffer-2,classe_onda-3,CARTON_M,instancia-3_classe_onda-3_tipo_buffer-2_sku-11...
154703,NaN,300.0,512.0,23.0,0.125392,0.075995,2024-06-14 11:17:56.217,0.5,0.005706,1,...,loja-104,rota-62,caixa-11283,sku-11930,itempai-3234,cor-718,tipo_buffer-2,classe_onda-3,CARTON_M,instancia-3_classe_onda-3_tipo_buffer-2_sku-11...
154704,NaN,300.0,512.0,23.0,0.125392,0.075995,2024-06-14 11:17:56.217,0.5,0.005706,1,...,loja-104,rota-62,caixa-11299,sku-11930,itempai-3234,cor-718,tipo_buffer-2,classe_onda-3,CARTON_M,instancia-3_classe_onda-3_tipo_buffer-2_sku-11...
154819,13.8,46.0,512.0,23.0,0.125392,0.075995,2024-06-14 11:17:56.217,0.5,0.005706,1,...,loja-106,rota-22,caixa-11406,sku-11930,itempai-3234,cor-718,tipo_buffer-2,classe_onda-3,CARTON_M,instancia-3_classe_onda-3_tipo_buffer-2_sku-11...
154856,NaN,300.0,512.0,23.0,0.125392,0.075995,2024-06-14 11:17:56.217,0.5,0.005706,1,...,loja-11,rota-62,caixa-10285,sku-11930,itempai-3234,cor-718,tipo_buffer-2,classe_onda-3,CARTON_M,instancia-3_classe_onda-3_tipo_buffer-2_sku-11...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157754,NaN,300.0,512.0,23.0,0.125392,0.075995,2024-06-14 11:17:56.217,0.5,0.005706,1,...,loja-84,rota-62,caixa-10996,sku-11930,itempai-3234,cor-718,tipo_buffer-2,classe_onda-3,CARTON_M,instancia-3_classe_onda-3_tipo_buffer-2_sku-11...
157901,NaN,70.0,512.0,23.0,0.125392,0.075995,2024-06-14 11:17:56.217,0.5,0.005706,1,...,loja-88,rota-61,caixa-11166,sku-11930,itempai-3234,cor-718,tipo_buffer-2,classe_onda-3,CARTON_M,instancia-3_classe_onda-3_tipo_buffer-2_sku-11...
157902,NaN,70.0,512.0,23.0,0.125392,0.075995,2024-06-14 11:17:56.217,0.5,0.005706,1,...,loja-88,rota-61,caixa-11198,sku-11930,itempai-3234,cor-718,tipo_buffer-2,classe_onda-3,CARTON_M,instancia-3_classe_onda-3_tipo_buffer-2_sku-11...
157903,NaN,70.0,512.0,23.0,0.125392,0.075995,2024-06-14 11:17:56.217,0.5,0.005706,1,...,loja-88,rota-61,caixa-11199,sku-11930,itempai-3234,cor-718,tipo_buffer-2,classe_onda-3,CARTON_M,instancia-3_classe_onda-3_tipo_buffer-2_sku-11...


In [46]:
df_output.query('chave_produtos == "instancia-3_classe_onda-3_tipo_buffer-2_sku-11930_CARTON_M"')[['item_pai','codigo_cor']].value_counts()

item_pai      codigo_cor
itempai-3234  cor-718       98
Name: count, dtype: int64

In [60]:
df_output.query('sku == "sku-11930"')[['item_pai','codigo_cor']].nunique()

item_pai      1
codigo_cor    1
dtype: int64

In [59]:
df_output[['sku','item_pai','codigo_cor']].value_counts()

sku        item_pai      codigo_cor
sku-1414   itempai-371   cor-783       267
sku-5684   itempai-1706  cor-36        235
sku-5685   itempai-1706  cor-36        212
sku-5682   itempai-1706  cor-36        212
sku-11930  itempai-3234  cor-718       208
                                      ... 
sku-12756  itempai-3667  cor-646         1
sku-12757  itempai-3668  cor-252         1
sku-6440   itempai-1920  cor-377         1
sku-6439   itempai-1919  cor-680         1
sku-9999   itempai-2921  cor-312         1
Name: count, Length: 15243, dtype: int64

In [57]:
df_output.groupby(['sku'])[['item_pai','codigo_cor']].nunique()

,item_pai,codigo_cor
sku,,
sku-1,1,1
sku-10,1,1
sku-100,1,1
sku-1000,1,1
sku-10000,1,1
...,...,...
sku-9995,1,1
sku-9996,1,1
sku-9997,1,1


O sku é um código único para item-pai e cor